In [2]:
#! pip install progressbar

In [3]:
from progressbar import ProgressBar

In [4]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
#from sklearn.neighbors import kneighbors_graph, NearestNeighbors
import scipy
import scipy.sparse as sparse
from scipy.stats import norm
import matplotlib.pyplot as plt

***Problem 1.*** Make a function StockVol to calibrate the stock volatility under geometric Brownian motion model. Input histoPrice is an array of 1-year historical prices. You can choose a
stock that does not pay dividends for simplicity. The function should return a number that is the
historical volatility of the stock.

In [5]:
def StockVol(histoPrice):
    """
    Compute the stock volatility under GBM using 1-year historical prices
    
    Inputs:
        histoPrice: an array of daily historical prices for one year
        
    Returns:
        histoVol: annualized historical volatility
    """
    
    logret = np.diff(np.log(histoPrice))
    sigma = np.sqrt(np.var(logret))
    histoVol = sigma*np.sqrt(252)  # annualize volatility
    
    return histoVol

***Problem 2.*** Make a function StockPath to generate n stock paths where n is one of the inputs,
as well as sigma which is the volatility of the stock. Other inputs needed (might not be limited to)
are S0: current stock price; T: terminal time in yearly unit; np: number of time periods; r: interest
rate; delta: continuous dividend yield of the stock.

In [6]:
def StockPath(n,sigma=0.2,S0=100,T=1,nump=252,r=0.01,delta = 0):
    """
    Generate n stock paths
    
    Inputs:
        n: number of paths generated
        sigma: volatility of the stock
        S0: current stock price
        T: terminal time in yearly unit
        nump: number of time periods
        r: interest rate
        delta: continuous dividend yield of the stock
        
    Returns:
        S: an array of stock paths
    """
    
    X = np.zeros((n,1+nump))
    X[:,0] = S0
    for i in range(len(X)):
        Z = np.random.normal(0, 1, nump)
        X[i,1:]=np.exp(sigma*np.sqrt(T/nump)*Z+(r-delta-sigma**2/2)*(T/nump))
    
    S = []   
    for i in range (n):
        S.append(np.cumprod(X[i,:]))
    
    return np.array(S)

***Problem 3.*** Make a function EurOptPrice that takes the stock paths to generate the European
put option price through Monte Carlo method. One input is n stock paths. The function should
return the discounted payoff vector, price, and variance.


In [7]:
def EurOptPrice(paths,K,r=0.01,T=1):
    """
    generate the European put option price through Monte Carlo method
    
    Inputs:
        paths: an array of stock paths
        K: strike price
        r: interest rate
        T: terminal time
        
    Returns:
        Payoff: discounted payoffs
        price: estimated price of the European put option
        variance: variance of discounted payoffs
        
    """
    
    Payoff = np.maximum(K-paths[:,-1],0)*np.exp(-r*T)
    price = np.mean(Payoff)
    variance = np.var(Payoff)
    
    return (Payoff,price,variance)
    

***Problem 4.*** Make a function AmeOptPrice that takes the stock paths to generate the American put option price without control variates. The function should return the discounted payoff
vector, price, and variance. In this part, you will need to implement some regression method and
you are required to do it by using machine learning or deep learning. Make sure to explain what
you did in the analysis file.

In [8]:
def AmeOptPrice(paths,K,r=0.01,T=1,nump = 252,delta = 0,sigma=0.2):
    """
    generate the American put option price without control variable
    
    Inputs:
        paths: an array of stock paths
        K: strike price
        r: interest rate
        T: terminal time
        nump: number of periods
        
    Returns:
        Payoff: discounted payoffs
        price: estimated price of the European put option
        variance: variance of discounted payoffs
        
    """
    deltaT = T/nump
    P = np.maximum(K-paths,0)  # payoffs if early exercise
    H = np.zeros(paths.shape)  # holding value
    V = np.zeros(paths.shape)  # value of the option
    
    H[:,-1] = P[:,-1]
    V[:,-1] = P[:,-1]
    
    # compute the expected payoff at termial time given S_(T-1) using one step monte carlo
    tmp = paths[:,-2]
    for i in range(len(paths)):
        tmp_Price = StockPath(100,sigma,tmp[i],deltaT,1,r,delta)
        tmp_payoff = np.maximum(K-tmp_Price[:,-1],0)*np.exp(-r*deltaT)
        H[i,-2] = np.mean(tmp_payoff)
    V[:,-2] = np.maximum(P[:,-2], H[:,-2])  # value of the option at t = T-1
    
    rf = RandomForestRegressor(n_estimators=30, n_jobs=-1)  #Define Random Forest Regressor 
    
    pbar = ProgressBar()
    for i in pbar(range(2,len(V[0]))):
        X = paths[:,-i].reshape(-1,1)
        Y = V[:,-i].reshape(-1,1)
        
        reg = rf.fit(X, Y.ravel())  # Polynomial regression (degree = 5)
        
        tmp = paths[:,-i-1]
        for j in range(len(paths)):
            tmp_Price = StockPath(100,sigma,tmp[j],deltaT,1,r,delta)
            tmp_V = rf.predict(tmp_Price.reshape(-1,1))*np.exp(-r*deltaT)
            H[j,-i-1] = np.mean(tmp_V)
        V[:,-i-1] = np.maximum(P[:,-i-1], H[:,-i-1])
 
    #Determine the optimal stopping time and payoffs
    Payoff = [0]*len(P)
    for i in range(len(P)):
        idx = np.where(P[i,:]> H[i,:])[0]
        if(len(idx) == 0):
            Payoff[i] = V[i,-1]*np.exp(-r*T)
        else:
            Payoff[i] = V[i,idx[0]]*np.exp(-r*idx[0]*deltaT)

    price = np.mean(Payoff)
    variance = np.var(Payoff)
    
    return(Payoff, price, variance)    

***Problem 5.*** Make a function ContVariate to implement the control variates method. Note that
this part is independent of the prices you computed. You should be able to apply this function to
any vectors and estimations.

In [9]:
def ContVariate(y,x,mu_x):
    """
    Implement the control variates method
    
    Inputs:
        y: an array of samples with unknown mean
        x: an array of samples with known or estimated mean
        mu_x: mean of random variable X
        
    Returns:
        y_hat: estimated mean of y
        y_hatVar: variance of the estimator
        
    """
    
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    y_var = np.var(y)
    corr = np.corrcoef(x,y)[0,1]
    
    beta = corr/np.var(x)
    y_hat = y_bar+beta*(mu_x-x_bar)
    y_hatVar = (np.var(y)/len(y))*(1-corr**2)
    
    return(y_hat,y_hatVar)

In [10]:
def BSput(S0, K, T, r, sigma,q):
    """
    Compute true price of the European put using BS formula
    
    Inputs:
        S0: current stock price
        K: strike price
        T: termial time
        r: interest rate
        sigma: volatility
        q: continuous dividend
        
    Output:
        price: price of the option
    """
    
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0 / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    price = (K * np.exp(-r * T) * norm.cdf(-d2, 0.0, 1.0) - S0 * norm.cdf(-d1, 0.0, 1.0))
    
    return price

## Test all functions 

In [36]:
S = StockPath(1000,0.2,100,1,12,0.01,delta = 0)

In [37]:
Payoff,price,variance = EurOptPrice(S,95,0.01,1)

In [38]:
price

5.093902051427781

In [39]:
variance

68.31186099061863

In [40]:
Payoff1,price1,variance1= AmeOptPrice(paths=S,K=95,r=0.01,T=1,nump = 12)

100% |########################################################################|


In [41]:
price1

5.108520888470472

In [42]:
variance1

54.226729148052485

In [19]:
truePrice = BSput(100, 95, 1, 0.01, 0.2, 0)

In [29]:
truePrice

5.114944153135006

In [43]:
ContVariate(Payoff1,Payoff,truePrice)

(5.108798204975928, 0.010274708113919779)

In [21]:
price1

6.014122159844128

In [297]:
variance1

35.49840650922919